# 

In [1]:
#!pip install torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:04m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 6.4 MB/s eta 0:00:000m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 5.1 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 6.8 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 3.4 MB/s eta 0:00:006.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:00:03m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 4.9 MB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.2 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 6.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.2 MB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━

In [9]:
import gc
import numpy as np
import matplotlib.pyplot as plt

import mplt_style
plt.style.use(mplt_style.style1)
import iMANGA_functions as iMANGA

from astropy.io import fits
from astropy.wcs import WCS # World coordinate system

from spectral_cube import SpectralCube    
from spectral_cube import LazyMask
import spectral_cube

import os

import copy


from astropy.io import fits
from pathlib import Path
from astropy.visualization import simple_norm

from matplotlib import rc#plt is broken
rc('text', usetex = False)


#############################
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms


#rectangular cropping function
#%run "/media/ben/home/ben/Documents/Grad Stuff/MM data/Simplified/2025/0 Imports and functions.ipynb"
%run "0_Imports.ipynb"

%matplotlib inline
#!pip show matplotlib
# imports a function called Crop_Nans
# imports the noise function "Noise_Matching"

For each image, simplify by creating a moment 0 map (which simulates a continuum image), then create a convolution with the assosiated true inclination value we are trying to reach

use the inner 256x256 pixels of the image for consistency

In [ ]:
# Create convolution and assosiated Inclination Value

In [32]:
folder = Path("Moment_Maps/")
########################################### Load
VAC_path = "Vac_Path/iMaNGA_vac.fits"
VAC =  fits.open(VAC_path, mode='denywrite')

snap=98

incl_cols = {"inc_morph_petr", "inc_morph_sersic", "inc_kin",
             "CosInclination", "inc_morph"}  # include a few possible names


#################
##############3 Gather the inclinations, then create a convolution pair with it and the data
################
for file in folder.glob("*.fits"):

    ########################################
    print(file)
    #ID
    ID = str(file)[12:len(str(file))-14] 
    print(ID)



    


    
    # find any table HDUs that contain one of the inclination columns
    printed_any = False

    galaxy_id_VAC = np.where(VAC[1].data['TNG_snap_id']==str(snap)+"-"+str(ID))[0][0] # index of the galaxy in the iMaNGA VAC


    #print(ID,galaxy_id_VAC)
    """
    ghhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh    ghhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
    ghhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
    ghhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
    ghhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
    ghhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
    USE THIS TO GET THE ACTUAYL VALUE AND PRINT IT
    for some stupid reaosn, the snap id is in the 1st header but all the data is in the 2nd hdu, so i cannot easily get them both at the same time.
    """
    data = VAC[1].data[galaxy_id_VAC]



    #go throuhgh the multiple HDUs in the vac (they contain diferent measurements, 2 contains the inclinations i think)
    for idx, hdu in enumerate(VAC):
        data = getattr(hdu, "data", None)
        #print(data)

        
        if data is None:
            continue

        data = data[galaxy_id_VAC-1:galaxy_id_VAC+1]
        #print(data)

        #galaxy_id_VAC = np.where(VAC[1].data['TNG_snap_id']==str(snap)+"-"+str(i))[0][0] # index of the galaxy in the iMaNGA VAC
        

    
        
        # if it's a recarray / table-like:
        names = [n.upper() for n in data.dtype.names] if data.dtype.names else []
        found = [n for n in names if n.lower() in {c.lower() for c in incl_cols}]
        
        if found:
            #print(names,found)
            print(f"\nHDU #{idx} contains inclination columns: {found}")
            # print header of the table and then the values
            # print a short header line
            header_cols = []
            header_cols = ["TNG_snap_id"] + found if "TNG_snap_id".upper() in names else found
            row = data[1]
            out = []
            for col in header_cols:
                # some FITS tables use uppercase names
                try:
                    out.append(str(row[col]))
                except Exception:
                    # try case-insensitive
                    matching = [n for n in data.dtype.names if n.lower()==col.lower()]
                    out.append(str(row[matching[0]]) if matching else "N/A")
            print(" | ".join(out))
            printed_any = True
            INC_MORPH_PETR, INC_MORPH_SERSIC, INC_KIN = out[0],out[1],out[2]



    ###########3###########3###########3###########3
    ###########3
    ########### Create a convolution of the image
    ############
    #Moment_0  = SpectralCube.read(file)
    #Moment_0  = SpectralCube.read(file)
    
    Moment_0  = spectral_cube.Projection.from_hdu(fits.open(file)) # For a 2d cube

    data_array = Moment_0.hdu.data

    # re shape
    outshape = [256,256]
    data_array = make_256(data_array,outsize = outshape)

    #padd iff  needed
    if len(data_array[0])<outshape[0] or len(data_array[1]) < outshape[1]:
        data_array = pad_to_256(data_array, pad_value=0, outsize = outshape)


    ##### FOrce the byte order to change for pytorch
    if not data_array.dtype.isnative:
        data_array = data_array.byteswap().newbyteorder()


    # Also force float32 so ToTensor works smoothly
    data_array = data_array.astype(np.float32)
    #################################################################

    #From torch transforms: 
    transform = transforms.Compose([
        transforms.ToTensor(),               # converts HxWxC numpy or PIL -> CxHxW torch.FloatTensor in [0,1]
        #transforms.Normalize((0.5,), (.5,)) # normalize channels (mean, std); adjust to match data
        transforms.Normalize(np.mean(data_array), np.std(data_array)) # normalize channels (mean, std); adjust to match data
    ])
    


    # Example: if you have a numpy array image
    # (grayscale: shape (H,W), or RGB: shape (H,W,3))
    
    # Convert + normalize
    normailization_tensor_img = transform(data_array)  # now a torch.FloatTensor

    data_pair = [[normailization_tensor_img],[INC_MORPH_PETR, INC_MORPH_SERSIC, INC_KIN]]
    #save_name = str(ID)+"_Convolution_Pair.npy"
    import pickle
    
    save_name = "Convolution_Pairs/"+str(ID) + "_Convolution_Pair.pkl"
    
    # Save
    with open(save_name, "wb") as f:
        pickle.dump(data_pair, f)
    #np.save(save_name,data_pair)

    """
    nn.Conv2d(1, 32, 3, 1)   # input=1 channel, output=32, kernel=3, donwsample = 1 (none)
    
    # Batch of 1, grayscale image 5x5
    x = torch.randn(1, 1, 5, 5)
    
    # Kernel: 1 output channel, 1 input channel, size 3x3
    w = torch.randn(1, 1, 3, 3)
    
    # Apply convolution
    y = F.conv2d(x, w, stride=1, padding=1)
    
    print("Input shape:", x.shape)
    print("Weight shape:", w.shape)
    print("Output shape:", y.shape)
    """


Moment_Maps/345537_Moment_0.fits
345537

HDU #2 contains inclination columns: ['INC_MORPH_PETR', 'INC_MORPH_SERSIC', 'INC_KIN']
29.41721 | 26.897617 | 43.56
Moment_Maps/682055_Moment_0.fits
682055

HDU #2 contains inclination columns: ['INC_MORPH_PETR', 'INC_MORPH_SERSIC', 'INC_KIN']
82.365005 | 87.63621 | 60.64
Moment_Maps/573973_Moment_0.fits
573973

HDU #2 contains inclination columns: ['INC_MORPH_PETR', 'INC_MORPH_SERSIC', 'INC_KIN']
68.912094 | 69.14864 | 86.31
Moment_Maps/731178_Moment_0.fits
731178

HDU #2 contains inclination columns: ['INC_MORPH_PETR', 'INC_MORPH_SERSIC', 'INC_KIN']
35.04797 | 32.964176 | 42.88
Moment_Maps/705626_Moment_0.fits
705626

HDU #2 contains inclination columns: ['INC_MORPH_PETR', 'INC_MORPH_SERSIC', 'INC_KIN']
57.70046 | 39.650215 | 39.43
Moment_Maps/118206_Moment_0.fits
118206

HDU #2 contains inclination columns: ['INC_MORPH_PETR', 'INC_MORPH_SERSIC', 'INC_KIN']
35.734867 | 24.27798 | 70.8
Moment_Maps/118303_Moment_0.fits
118303

HDU #2 contains in

# Now the images are in a state where convolution can be applied and the weights of the convolution are optimized in the network.
# A continuous regression is applied to try and hit the tru kinematic inclination value of the galaxy.